<a href="https://colab.research.google.com/github/murillofnc22/labs-bairesdev-ml-training/blob/main/SistemaReconhecimentoFacial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Instalação de dependências
!pip install -q tensorflow opencv-python matplotlib numpy dlib
!apt-get -qq install -y libsm6 libxext6 libxrender-dev

!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (dlib)
--2025-09-16 02:09:49--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-09-16 02:09:50--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.

In [47]:
# Importações necessárias
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_distances
import pickle
import dlib

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Configurações
IMAGE_SIZE = (160, 160)
DETECTION_THRESHOLD = 0.7
RECOGNITION_THRESHOLD = 0.5

TF version: 2.19.0
GPU is available


In [48]:
# 1. Funções para detecção facial (usando OpenCV apenas)
class FaceDetector:
    def __init__(self):
        # Carregar modelos Haar Cascades do OpenCV
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.eye_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_eye.xml'
        )

    def detect_faces(self, image):
        """Detecta faces usando Haar Cascades"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detectar faces
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        return faces

# 2. Funções para extração de características faciais (simplificado)
class FaceFeatureExtractor:
    def __init__(self):
        self.detector = dlib.get_frontal_face_detector()
        dlib.DLIB_USE_CUDA = False
        self.shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        self.face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

    def preprocess_face(self, image):
        dets = self.detector(image, 1)
        if len(dets) == 0:
            return None, None
        # pega a maior face
        face_rect = max(dets, key=lambda r: r.width() * r.height())
        shape = self.shape_predictor(image, face_rect)
        return face_rect, shape

    def get_embedding(self, image):
        try:
            face_rect, shape = self.preprocess_face(image)
            if face_rect is None:
                return np.zeros(128)
            embedding = self.face_rec_model.compute_face_descriptor(image, shape)
            return np.array(embedding)
        except Exception as e:
            print(f"Erro na extração de embedding: {e}")
            return np.zeros(128)

# 3. Sistema de reconhecimento facial
class FaceRecognizer:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor
        self.encoder = None
        self.classifier = None
        self.embeddings = None
        self.labels = None
        self.DISTANCE_THRESHOLD = 0.6  # ajuste fino

    def extract_faces(self, image):
        """Extrai faces de uma imagem"""
        faces = []
        face_coords = []

        try:
            detected_faces = self.detector.detect_faces(image)

            for (x, y, w, h) in detected_faces:
                # Garantir que as coordenadas estão dentro da imagem
                x, y = max(0, x), max(0, y)
                w, h = min(w, image.shape[1] - x), min(h, image.shape[0] - y)

                if w > 20 and h > 20:  # Mínimo tamanho para ser uma face
                    face_img = image[y:y+h, x:x+w]
                    faces.append(face_img)
                    face_coords.append((x, y, w, h))

        except Exception as e:
            print(f"Erro na extração de faces: {e}")

        return faces, face_coords

    def train_from_directory(self, dataset_path):
      embeddings = []
      labels = []

      for person_name in os.listdir(dataset_path):
          person_dir = os.path.join(dataset_path, person_name)
          if not os.path.isdir(person_dir):
              continue

          for image_name in os.listdir(person_dir):
              image_path = os.path.join(person_dir, image_name)
              image = cv2.imread(image_path)

              if image is None:
                  print(f"Erro ao carregar {image_path}")
                  continue

              embedding = self.feature_extractor.get_embedding(image)

              if embedding is not None and not np.all(embedding == 0):
                  embeddings.append(embedding)
                  labels.append(person_name)

      embeddings = np.array(embeddings)
      labels = np.array(labels)

      if len(embeddings) == 0:
          print("Nenhuma face encontrada no dataset!")
          return

      # 🔹 Encoder para transformar labels em números
      self.encoder = LabelEncoder()
      encoded_labels = self.encoder.fit_transform(labels)

      # 🔹 Classificador (opcional, ainda podemos usar junto)
      self.classifier = SVC(kernel='linear', probability=True)
      self.classifier.fit(embeddings, encoded_labels)

      # 🔹 Guardar embeddings e labels originais
      self.embeddings = embeddings
      self.labels = labels

      print(f"Treinamento concluído com {len(labels)} faces de {len(set(labels))} pessoas.")

    def recognize_faces(self, embeddings):
        results = []
        for embedding in embeddings:
            if embedding is None or np.all(embedding == 0):
                results.append(("Desconhecido", 0.0))
                continue

            # --- Reconhecimento por distância ---
            distances = cosine_distances([embedding], self.embeddings)[0]
            min_idx = np.argmin(distances)
            min_dist = distances[min_idx]

            if min_dist < self.DISTANCE_THRESHOLD:
                label = self.labels[min_idx]
                confidence = 1 - min_dist
                results.append((label, confidence))
            else:
                results.append(("Desconhecido", 1 - min_dist))

        return results

# 4. Funções de utilidade
def draw_faces(image, results):
    """Desenha retângulos e labels nas faces detectadas"""
    image_with_boxes = image.copy()

    for result in results:
        x, y, w, h = result['coordinates']
        label = result['label']
        confidence = result['confidence']

        # Desenhar retângulo
        color = (0, 255, 0) if label != "Desconhecido" else (0, 0, 255)
        cv2.rectangle(image_with_boxes, (x, y), (x+w, y+h), color, 2)

        # Desenhar label
        label_text = f"{label}: {confidence:.2f}" if confidence > 0 else label
        cv2.putText(image_with_boxes, label_text, (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return image_with_boxes

def test_with_sample_image(recognizer, image_path=None):
    """Testa o reconhecedor com uma imagem de exemplo"""
    if image_path is None:
        # Criar imagem de teste simples
        test_image = np.ones((300, 400, 3), dtype=np.uint8) * 255
        cv2.rectangle(test_image, (50, 50), (150, 150), (100, 100, 200), -1)
        cv2.rectangle(test_image, (200, 80), (300, 180), (200, 100, 100), -1)
        image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    else:
        # Carregar imagem real
        test_image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    # Reconhecer faces
    results = recognizer.recognize_faces(image_rgb)

    # Desenhar resultados
    result_image = draw_faces(test_image, results)
    result_image_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

    # Mostrar resultado
    plt.figure(figsize=(12, 8))
    plt.imshow(result_image_rgb)
    plt.title('Resultado do Reconhecimento Facial')
    plt.axis('off')
    plt.show()

    print("Resultados do reconhecimento:")
    for i, result in enumerate(results):
        print(f"Face {i+1}: {result['label']} (confiança: {result['confidence']:.2f})")

    return results

In [49]:
# 1. Criar extrator + reconhecedor
extractor = FaceFeatureExtractor()
recognizer = FaceRecognizer(extractor)

# 2. Treinar a partir do dataset (pasta com subpastas por pessoa)
recognizer.train_from_directory("/content/celebrity_faces_dataset")

# 3. Testar com nova imagem
test_img = cv2.imread('/content/celebrity_faces_dataset/robert_downey_jr/001_a51bb26a.jpg')
test_embedding = extractor.get_embedding(test_img)
result = recognizer.recognize_faces([test_embedding])

print(result)  # → [('PessoaX', 0.88)] ou ('Desconhecido', 0.45)


Erro na extração de embedding: Error while calling cudaOccupancyMaxPotentialBlockSize(&num_blocks,&num_threads,K) in file /tmp/.tmpH7KQ6H/sdists-v9/pypi/dlib/19.24.6/VcByn2lOSjIQgvHLeoPVu/src/dlib/cuda/cuda_utils.h:186. code: 222, reason: the provided PTX was compiled with an unsupported toolchain.
Erro na extração de embedding: Error while calling cudaOccupancyMaxPotentialBlockSize(&num_blocks,&num_threads,K) in file /tmp/.tmpH7KQ6H/sdists-v9/pypi/dlib/19.24.6/VcByn2lOSjIQgvHLeoPVu/src/dlib/cuda/cuda_utils.h:186. code: 222, reason: the provided PTX was compiled with an unsupported toolchain.
Erro na extração de embedding: Error while calling cudaOccupancyMaxPotentialBlockSize(&num_blocks,&num_threads,K) in file /tmp/.tmpH7KQ6H/sdists-v9/pypi/dlib/19.24.6/VcByn2lOSjIQgvHLeoPVu/src/dlib/cuda/cuda_utils.h:186. code: 222, reason: the provided PTX was compiled with an unsupported toolchain.
Erro na extração de embedding: Error while calling cudaOccupancyMaxPotentialBlockSize(&num_blocks,&n

KeyboardInterrupt: 